## 무신사 스탠다드 리뷰 크롤링

**환경설정**

In [ ]:
#필요하면 설치하기
#!pip install html_table_parser
#!pip install --upgrade beautifulsoup4

**변경 사항**
- 리뷰 개수가 100개 이하인 상품도 모두 긁어옴
    - 100개 이하 : 모두, 100개 이상 : 100개
- 원하는 page에 대한 url을 긁어올 필요 없이, page(n)을 수정해주면 됨

**환경 설정**

In [2]:
import os
import time
import re
import math

import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from html_table_parser import parser_functions as parser

import collections
collections.Callable = collections.abc.Callable

import warnings
warnings.filterwarnings("ignore")

#추가 환경설정
from webdriver_manager.chrome import ChromeDriverManager

In [3]:
def get_review_content(driver, review_each_num):
    '''
    크롤링해서 내용 추출하는 함수 
    - 상품 클릭한 상태에서 해당 페이지 리뷰 10개 크롤링
    '''
    html = driver.page_source
    soup = BeautifulSoup(html,'lxml')
    
    # user, gender,height,weight,item, size,content, evaluation(size_eval,bright_eval,color_eval,thick_eval)
    user_list = []
    gender_list =[]
    height_list = []
    weight_list =[]
    item_list = []
    size_list = []
    star_list= []
    content_list = []
    size_eval_list =[]
    bright_eval_list =[]
    color_eval_list =[]
    thick_eval_list =[]
    
    for i in range(review_each_num):
        # profile(gender,height,weight)
        # <p class="review-profile__body_information">남성, 177cm, 85kg</p>
        try:
            profile_before = soup.find_all('p','review-profile__body_information')
            profile_after = profile_before[i].text.split(',')
            gender = profile_after[0]
            height = profile_after[1]
            weight = profile_after[2]
        except:
            gender = ''
            height = ''
            weight = ''
            
        # user :<p class="review-profile__name">LV 2 뉴비_95f88e16</p>           
        # item: #/<a href="https://www.musinsa.com/app/goods/1231416/0" class="review-goods-information__name">테이퍼드 히든 밴딩 크롭 슬랙스 [더스티 베이지]</a>
        # size :<span class="review-goods-information__option">
        # content
        try:
            user = soup.find_all('p','review-profile__name')[i].text
            item = soup.find_all('a','review-goods-information__name')[i].text
            size = soup.find_all('span', 'review-goods-information__option')[i].text.strip().replace('\n','') # '/n' 없애고 추출하기
            content = soup.find_all('div','review-contents__text')[i].text
        except:
            user = ''
            item = ''
            size = ''
            content = ''
            
        #star
        #->별 5개일때:<span class="review-list__rating__active" style="width: 100%"></span>
        #->별 4개일때:<span class="review-list__rating__active" style="width: 80%"></span>
        try:
            stars = driver.find_elements_by_xpath('//*[@id="reviewListFragment"]/div['+str(i+1)+']/div[3]/span/span/span')
            for j in stars:
                a =j.get_attribute('style')
                if a[7:9]=='20':
                    star = 1
                elif a[7:9]=='40':
                    star = 2
                elif a[7:9]=='60':
                    star = 3
                elif a[7:9]=='80':
                    star = 4
                else:
                    star = 5
        except:
            star = ''
      
        # evaluation
        try:
            evaluation = soup.find_all('div', 'review-evaluation')
            size_eval = evaluation[i].find_all('span')[0].text
            bright_eval = evaluation[i].find_all('span')[1].text
            color_eval = evaluation[i].find_all('span')[2].text
            thick_eval = evaluation[i].find_all('span')[3].text
        except:
            size_eval = ''
            bright_eval = ''
            color_eval = ''
            thick_eval = ''
        
        user_list.append(user)
        gender_list.append(gender)
        height_list.append(height)
        weight_list.append(weight)
        item_list.append(item)
        size_list.append(size)
        content_list.append(content)
        star_list.append(star)
        size_eval_list.append(size_eval)
        bright_eval_list.append(bright_eval)
        color_eval_list.append(color_eval)
        thick_eval_list.append(thick_eval)
    
    return user_list, gender_list, height_list, weight_list, item_list, size_list, content_list, star_list, size_eval_list, bright_eval_list, color_eval_list, thick_eval_list


In [4]:
def get_item_content(driver, reviewNum):
    lastPage = int(reviewNum/10)+1 # 리뷰 100개 이하일 때 마지막 페이지
    lastPage_review_num = int(reviewNum%10) # 리뷰 100개 이하일 때 마지막 페이지 리뷰 수
    
    # 첫 페이지
    page_cnt = 1
    print(page_cnt, end=" ")
    if page_cnt == lastPage:
        user_list, gender_list, height_list, weight_list, item_list, size_list, content_list, star_list, size_eval_list, bright_eval_list, color_eval_list, thick_eval_list = get_review_content(driver, lastPage_review_num)
    else:
        user_list, gender_list, height_list, weight_list, item_list, size_list, content_list, star_list, size_eval_list, bright_eval_list, color_eval_list, thick_eval_list = get_review_content(driver, 10)

    for i in range(2):
        for j in range(4, 9 ,1):
            
            if page_cnt >= 10: # 리뷰 100개만 긁어야하니 stop
                break
            if page_cnt == lastPage: # 리뷰가 100개 이하면 마지막 페이지에서 더 이상 못 넘기므로 stop
                break
            
            try:
                if j in [4,5,6,7]:
                    driver.find_element_by_css_selector('#reviewListFragment > div.nslist_bottom > div.pagination.textRight > div > a:nth-child(' + str(int(j)) + ')').send_keys(Keys.ENTER)
                elif j == 8:
                    driver.find_element_by_css_selector('#reviewListFragment > div.nslist_bottom > div.pagination.textRight > div > a.fa.fa-angle-right.paging-btn.btn.next').send_keys(Keys.ENTER)
                page_cnt += 1
                print(page_cnt, end=" ")
            except:
                print('{}, {} button click except'.format(i, j))
            time.sleep(2)

            
            # 첫 페이지 이외의 페이지 정보 첫 페이지 리스트에 추가
            # 마지막 페이지는 10개가 안될 수 있어 조건 부여
            if page_cnt == lastPage:
                user_list_1, gender_list_1, height_list_1, weight_list_1, item_list_1, size_list_1, content_list_1, star_list_1, size_eval_list_1, bright_eval_list_1, color_eval_list_1, thick_eval_list_1 = get_review_content(driver, lastPage_review_num)
            else:
                user_list_1, gender_list_1, height_list_1, weight_list_1, item_list_1, size_list_1, content_list_1, star_list_1, size_eval_list_1, bright_eval_list_1, color_eval_list_1, thick_eval_list_1 = get_review_content(driver, 10)
            
            user_list.extend(user_list_1)
            gender_list.extend(gender_list_1)
            height_list.extend(height_list_1)
            weight_list.extend(weight_list_1)
            item_list.extend(item_list_1)
            size_list.extend(size_list_1)
            content_list.extend(content_list_1)
            star_list.extend(star_list_1)
            size_eval_list.extend(size_eval_list_1)
            bright_eval_list.extend(bright_eval_list_1)
            color_eval_list.extend(color_eval_list_1)
            thick_eval_list.extend(thick_eval_list_1)
            
            time.sleep(2)

    item_review_df = pd.DataFrame({'user':user_list,
                                   'gender':gender_list,
                                   'height':height_list,
                                   'weight':weight_list,
                                   'item':item_list,
                                   'size':size_list,
                                   'star':star_list,
                                   'content':content_list,
                                   'size_eval':size_eval_list,
                                   'bright_eval':bright_eval_list,
                                   'color_eval':color_eval_list,
                                   'thick_eval':thick_eval_list})
            
    return item_review_df
    

def get_data(driver, page_url, start_item, item_cnt):
    final_df = pd.DataFrame() # item 90개 정보 담은 최종 DataFrame
    
    # 후기순 90개 자동으로 클릭하기

    for t in tqdm(range(start_item, item_cnt+1,1), desc = "Description"):
        print("Item {} : ".format(t), end = " ")
        ## 상품 클릭
        try: 
            # #searchList > li:nth-child(1) > div.li_inner > div.list_img > a > img
            # #searchList > li:nth-child(90) > div.li_inner > div.list_img > a > img
            driver.find_element_by_css_selector('#searchList > li:nth-child(' + str(t) + ') > div.li_inner > div.list_img > a').click()
            time.sleep(2)
        except:
            print("item click issue")
            continue
        
        ## 팝업 처리
        # 일단 빈공간 클릭 -> 창이 랜덤으로 뜨기 때문에 빈곳을 클릭하면 팝업이 뜨는 경우가 있음
        driver.find_element_by_xpath('//*[@id="product_order_info"]/div[1]/h4')
        time.sleep(2)
        
        try:
            #무신사쿠폰 팝업창: 해당 팝업이 뜨면 나머지 선택 안됨
            driver.find_element_by_xpath('/html/body/div/div/div/button').click()
            time.sleep(2)
            #입고지연팝업창: 삭제안해도 나머지 구동 가능
            #driver.find_element_by_xpath('//*[@id="divpop_goods_niceghostclub_8451"]/form/button[2]').click()
            #무신사회원혜택 팝업창 :삭제안해도 나머지 구동가능
            #driver.find_element_by_xpath('//*[@id="page_product_detail"]/div[3]/div[23]/div/a[1]/img').click()
        except:
            pass
            
        ## 사이즈표 추출
        try:
            html = driver.page_source
            soup = BeautifulSoup(html,'html.parser')
            figure = soup.find('table',{'class':'table_th_grey'})
            p = parser.make2d(figure)
            figure_df = pd.DataFrame(data = p[1:],columns = p[0])
            figure_df.drop([0,1],inplace = True)
        except:
            print("사이즈표 오류발생")
            
            
        ## 리뷰개수 체크
        try:
            reviewNum = driver.find_element_by_xpath('//*[@id="estimate_style"]')
            reviewNum = reviewNum.text
            reviewNum = re.sub(r'[^0-9]','',reviewNum)
            reviewNum = int(reviewNum)
        except:
            print("리뷰개수 오류발생")
            reviewNum = 0
        
        ## 각 상품에 해당하는 리뷰 추출
        item_review_df = get_item_content(driver, reviewNum)
        
        merge_df = pd.merge(item_review_df, figure_df, how = 'left', left_on = 'size', right_on = 'cm')
        print(merge_df.shape)
        
        final_df = pd.concat([final_df, merge_df])
        
        #뒤로가기
        driver.get(page_url)
        time.sleep(2)

    return final_df

In [5]:
if __name__=='__main__':
    page = 2
    page_url = 'https://www.musinsa.com/categories/item/001004?d_cat_cd=001004&brand=&list_kind=small&sort=emt_high&sub_sort=&page={}&display_cnt=90&group_sale=&exclusive_yn=&sale_goods=&timesale_yn=&ex_soldout=&kids=&color=&price1=&price2=&shoeSizeOption=&tags=&campaign_id=&includeKeywords=&measure='.format(str(page))
    
    options = webdriver.ChromeOptions()
    options.add_experimental_option("excludeSwitches", ["enable-logging"])
    options.add_experimental_option("detach", True) # 마지막에 창 안닫히게
    
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
    #driver = webdriver.Chrome("./chromedriver.exe", options=options) #원래 driver 코드
    
    driver.get(page_url)
    time.sleep(3)

    start = time.time()

    start_item = 1
    item_cnt = 11
    # re_cnt = 100  # 기준으로 잡은 스타일후기리뷰 개수 (우선 생략)
    
    # 크롤링 진행
    final_df = get_data(driver, page_url, start_item, item_cnt)

    # 창 종료
    driver.quit()  
    
    # final_df.drop_duplicates(subset = None, keep = 'first', inplace = True, ignore_index = True) # 중복아이템 제거 (나중에 합치고 체크)
    print('최종개수:',final_df.shape) #최종 data: final_df
    
    end = time.time()
    print("{:.5f} sec".format(end-start))

    # csv로 저장
    final_df.to_csv("hood_2_1_11page.csv", encoding="UTF-8", index=False)



Description:   0%|                                                                              | 0/11 [00:00<?, ?it/s]

Item 1 :  1 2 3 4 5 6 7 8 9 10 (100, 17)



Description:   9%|██████▎                                                              | 1/11 [04:33<45:38, 273.81s/it]

Item 2 :  1 2 3 4 5 6 7 8 9 10 (100, 17)



Description:  18%|████████████▌                                                        | 2/11 [08:12<38:35, 257.32s/it]

...

Item 10 :  1 2 3 4 5 6 7 8 9 10 (100, 17)



Description:  91%|█████████████████████████████████████████████████████████████▊      | 10/11 [37:11<03:35, 215.11s/it]

Item 11 :  1 2 3 4 5 6 7 8 9 10 (100, 17)


Description: 100%|████████████████████████████████████████████████████████████████████| 11/11 [41:01<00:00, 223.77s/it]


최종개수: (1100, 17)
2472.06932 sec


- page2(12-30) - 1,800개 - 2,653초 - 크롤링완
- page2(31-40) - 1,000개 - 1,985초 - 크롤링완
- page2(41-70) - 3,000개 - 5,802초 - 크로링완
- page6   - 6,419개 - 9,689초 - 크롤링완
- page10 - 5,154개 - 7,317초 - 크롤링완
- page14 - 3,673개 - 6,197초 - 크롤링완
- page18 - 2,445개 - 7,433초 - 크롤링완

### 무신사 리뷰 크롤링
- 주은언니깃헙코드 수정버전

- BeautifulSoup 업데이트로 에러 발생
- 'AttributeError : module 'html5lib.treebuilders' has no attribute '_base''
- pip install --upgrade beautifulsoup4

In [ ]:
import warnings
warnings.filterwarnings("ignore")
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import re
import time
import pandas as pd

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from html_table_parser import parser_functions as parser

In [ ]:
#함수정의:검색어 조건에 따른 url생성
#무신사 스탠다드->후기순->벨트 및 양말 등 잡화제외 1위 상품
def musinsa_searching(word):
  url = "https://www.musinsa.com/app/goods/1149328" 
  return url

In [ ]:
from selenium.webdriver.common.keys import Keys
#가져오고싶은 상품 페이지 가져오기
#일단 후드-후기순-3번째페이지 링크 수기로 입력
url = 'https://www.musinsa.com/categories/item/001004?d_cat_cd=001004&brand=&list_kind=small&sort=emt_high&sub_sort=&page=3&display_cnt=90&group_sale=&exclusive_yn=&sale_goods=&timesale_yn=&ex_soldout=&kids=&color=&price1=&price2=&shoeSizeOption=&tags=&campaign_id=&includeKeywords=&measure='

from webdriver_manager.chrome import ChromeDriverManager
options = webdriver.ChromeOptions()
options.add_experimental_option('excludeSwitches', ['enable-logging'])

driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
driver.get(url)
time.sleep(5)

In [ ]:
#각 리뷰에서 아래의 값들을 추출하는 함수 정의
#user, gender,height,weight,item, size,content, evaluation(size_eval,bright_eval,color_eval,thick_eval)
user_list = []
gender_list =[]
height_list = []
weight_list =[]
item_list = []
size_list = []
star_list= []
content_list = []
size_eval_list =[]
bright_eval_list =[]
color_eval_list =[]
thick_eval_list =[]

#함수정의
def get_content(driver):
    #함수안에 html, soup 넣어놔야 페이지 넘어가서 바르게 긁어옴, 밖에 빼놓으면 첫페이지만 여러번 긁어진다.
    html = driver.page_source
    soup = BeautifulSoup(html,'lxml')
    for i in range(10):
    
    #한 페이지당 10개의 리뷰씩 긁어와야하므로
    #profile(gender,height,weight)
    #남성, 177cm, 85kg

        profile_before = soup.find_all('p','review-profile__body_information')
        profile_after = profile_before[i].text.split(',')
        try:
            gender = profile_after[0]
            height = profile_after[1]
            weight = profile_after[2]
        except:
            gender = ''
            height = ''
            weight = ''
#user :LV 2 뉴비_95f88e16

           
#item: #/테이퍼드 히든 밴딩 크롭 슬랙스 [더스티 베이지]
#size :
#content

        try:
            user = soup.find_all('p','review-profile__name')[i].text
            item = soup.find_all('a','review-goods-information__name')[i].text
            # '/n' 없애고 추출하기
            size = soup.find_all('span', 'review-goods-information__option')[i].text.strip().replace('\n','')
            content = soup.find_all('div','review-contents__text')[i].text
        except:
            user = ''
            item = ''
            size = ''
            content = ''
            
        #star
#->별 5개일때:
#->별 4개일때:
        stars = driver.find_elements_by_xpath('//*[@id="reviewListFragment"]/div['+str(i+1)+']/div[3]/span/span/span')
        try:
            for j in stars:
                a =j.get_attribute('style')
                if a[7:9]=='20':
                    star = 1
                elif a[7:9]=='40':
                    star = 2
                elif a[7:9]=='60':
                    star = 3
                elif a[7:9]=='80':
                    star = 4
                else:
                    star = 5
        except:
            star = ''
      

    #evaluation
        evaluation = soup.find_all('div', 'review-evaluation')
        try:
            size_eval = evaluation[i].find_all('span')[0].text
            bright_eval = evaluation[i].find_all('span')[1].text
            color_eval = evaluation[i].find_all('span')[2].text
            thick_eval = evaluation[i].find_all('span')[3].text
        except:
            size_eval = ''
            bright_eval = ''
            color_eval = ''
            thick_eval = ''

#user,gender,height,weight,item, size,content,star,size_eval,bright_eval,color_eval,thick_eval
        
        user_list.append(user)
        gender_list.append(gender)
        height_list.append(height)
        weight_list.append(weight)
        item_list.append(item)
        size_list.append(size)
        content_list.append(content)
        star_list.append(star)
        size_eval_list.append(size_eval)
        bright_eval_list.append(bright_eval)
        color_eval_list.append(color_eval)
        thick_eval_list.append(thick_eval)
 

In [ ]:
#버튼 누르기 함수정의
def move_next(driver):    
    for i in range(4):
        get_content(driver)
        #페이지 2,3,4,5 넘어가기
        driver.find_element_by_css_selector('#reviewListFragment > div.nslist_bottom > div.pagination.textRight > div > a:nth-child(' + 
                                            str(int(4) + int(i)) + ')').send_keys(Keys.ENTER)
        time.sleep(2)
    get_content(driver)
    
#그다음 화살표'>'버튼누르기: (6,7,8...)있는 페이지로 넘어가기   
def move_arrow(driver):
    driver.find_element_by_css_selector('#reviewListFragment > div.nslist_bottom > div.pagination.textRight > div > a.fa.fa-angle-right.paging-btn.btn.next').send_keys(Keys.ENTER)


**크롤링 시작**

In [ ]:
from selenium.webdriver.common.keys import Keys
#가져오고싶은 상품 페이지 가져오기
#일단 후드-후기순-n번째페이지 url 수기로 입력
url = 'https://www.musinsa.com/categories/item/001004?d_cat_cd=001004&brand=&list_kind=small&sort=emt_high&sub_sort=&page=7&display_cnt=90&group_sale=&exclusive_yn=&sale_goods=&timesale_yn=&ex_soldout=&kids=&color=&price1=&price2=&shoeSizeOption=&tags=&campaign_id=&includeKeywords=&measure='

from webdriver_manager.chrome import ChromeDriverManager
options = webdriver.ChromeOptions()
options.add_experimental_option('excludeSwitches', ['enable-logging'])
#알림창 닫는 옵션인데 안먹힘
#options.add_experimental_option('prefs', {'profile.default_content_setting_values.notifications' : 1})

driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
driver.get(url)
time.sleep(2)

In [ ]:
#후기순 90개 자동으로 클릭하기
#뒤로가기 참고: https://m.blog.naver.com/mathesis_time/221970036063
# #searchList > li:nth-child(1) > div.li_inner > div.list_img > a > img
# #searchList > li:nth-child(2) > div.li_inner > div.list_img > a > img
# #searchList > li:nth-child(90) > div.li_inner > div.list_img > a > img
#90개: 90
import math
import time

start = time.time()
math.factorial(100000)

#a값은 상품을 변경할때마다 변경해주어야함
a = 68
#상품개수에 따라 숫자 바꾸기
for i in range(67,90,1):
    driver.get(url)
    driver.find_element_by_css_selector('#searchList > li:nth-child(' +
                                            str(int(i)+int(1)) +
                                            ') > div.li_inner > div.list_img > a > img').click()
    time.sleep(2)
    try:
        driver.find_element_by_xpath('/html/body/div/div/div/button').click()
        driver.find_element_by_xpath('//*[@id="divpop_goods_niceghostclub_8451"]/form/button[2]').click()
    except:
        pass
    
    #사이즈표
    try:
        html = driver.page_source
        soup = BeautifulSoup(html,'html.parser')
        figure = soup.find('table',{'class':'table_th_grey'})
        p = parser.make2d(figure)
        figure_df = pd.DataFrame(data = p[1:],columns = p[0])
        figure_df.drop([0,1],inplace = True)
        print(figure_df)
    except:
        print("사이즈표 오류발생")
        
    #리뷰개수
    try:
        reviewNum = driver.find_element_by_css_selector('#estimate_style')
        reviewNum = reviewNum.text
        reviewNum = re.sub(r'[^0-9]','',reviewNum)
        reviewNum = int(reviewNum)
        print(reviewNum)
    except:
        print("리뷰개수 오류발생")
        

    user_list = []
    gender_list =[]
    height_list = []
    weight_list =[]
    item_list = []
    size_list = []
    star_list= []
    content_list = []
    size_eval_list =[]
    bright_eval_list =[]
    color_eval_list =[]
    thick_eval_list =[]
        
    #스타일후기리뷰 100개넘으면 진행,아니라면 진행 X
    if reviewNum >= 100:
        #크롤링
        #10:500개, 2:100개
        for i in range(2):
            try:
                move_next(driver)
                move_arrow(driver)
        #move_next(driver)
            except:
                time.sleep(2)

    else:
        driver.get(url)
        time.sleep(2)    
        
    globals()["df"+str(a)] = pd.DataFrame({'user':user_list,
                                           'gender':gender_list,
                                           'height':height_list,
                                           'weight':weight_list,
                                            'item':item_list,
                                            'size':size_list,
                                            'star':star_list,
                                            'content':content_list,
                                            'size_eval':size_eval_list,
                                            'bright_eval':bright_eval_list,
                                            'color_eval':color_eval_list,
                                            'thick_eval':thick_eval_list})          
    #사이즈표와 리뷰 merge
    time.sleep(2) 
    globals()["merge_df"+str(a)] = pd.merge(globals()["df"+str(a)],figure_df,how = 'left',left_on = 'size',right_on = 'cm')
    a = a + 1 
    
end = time.time()
print(f"{end - start:.5f} sec")

**크롤링 리뷰 개수 확인**

In [ ]:
print(merge_df1.shape)
print(merge_df2.shape)
print(merge_df3.shape)
print(merge_df4.shape)
print(merge_df5.shape)
print(merge_df6.shape)
print(merge_df7.shape)
print(merge_df8.shape)
print(merge_df9.shape)
print(merge_df10.shape)

In [ ]:
page7 = pd.concat([merge_df1,merge_df2,merge_df3,merge_df4,merge_df5,merge_df6,merge_df7,merge_df8,merge_df9,merge_df10,
                   merge_df11,merge_df12,merge_df13,merge_df14,merge_df15,merge_df16,merge_df17,merge_df18,merge_df19,merge_df20,
                   merge_df21,merge_df22,merge_df23,merge_df24,merge_df25,merge_df26,merge_df27,merge_df28,merge_df29,merge_df30,
                   merge_df31,merge_df32,merge_df33,merge_df34,merge_df35,merge_df36,merge_df37,merge_df38,merge_df39,merge_df40,
                   merge_df41,merge_df42,merge_df43,merge_df44,merge_df45,merge_df46,merge_df47,merge_df48,merge_df49,merge_df50,
                   merge_df51,merge_df52,merge_df53,merge_df54,merge_df55,merge_df56,merge_df57,merge_df58,merge_df59,merge_df60,
                   merge_df61,merge_df62,merge_df63,merge_df64,merge_df65,merge_df66,merge_df67,merge_df68,merge_df69,merge_df70,
                   merge_df71,merge_df72,merge_df73,merge_df74,merge_df75,merge_df76,merge_df77,merge_df78,merge_df79,merge_df80,
                   merge_df81,merge_df82,merge_df83,merge_df84,merge_df85,merge_df86,merge_df87,merge_df88,merge_df89,merge_df90,
                  ], axis=0)

**중복되는 관측치 제거**

In [ ]:
page7.info() #1-12 이 동일한 관측치들만 뽑아내기

In [ ]:
df = page7.iloc[:,[0,1,2,3,5,6,7,8,9,10,11]]
df.head(5)

In [ ]:
page7.reset_index(inplace=True, drop=True)
page7

In [ ]:
df.reset_index(inplace=True, drop=True)
df

In [ ]:
df1 = df.drop_duplicates(subset=None, keep='first', inplace=False)
df1

In [ ]:
page7_nodup = page7.iloc[df1.index, :]

In [ ]:
page7.to_csv('page7_product90.csv', encoding='utf-8')

In [ ]:
page7_nodup.to_csv('page7_product90_nodup.csv', encoding='utf-8')

### 무신사 리뷰 크롤링
- 12월 14일 기준 - 주은언니깃헙

In [ ]:
#환경설정
import warnings
warnings.filterwarnings("ignore")
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import re
import math
import pandas as pd

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from html_table_parser import parser_functions as parser

In [ ]:
#필요한 함수정의
user_list = []
gender_list =[]
height_list = []
weight_list =[]
item_list = []
size_list = []
star_list= []
content_list = []
size_eval_list =[]
bright_eval_list =[]
color_eval_list =[]
thick_eval_list =[]
#함수정의
def get_content(driver):
    #함수안에 html, soup 넣어놔야 페이지 넘어가서 바르게 긁어옴, 밖에 빼놓으면 첫페이지만 여러번 긁어진다.
    html = driver.page_source
    soup = BeautifulSoup(html,'lxml')
    for i in range(10):
    
    #profile(gender,height,weight)
    #<p class="review-profile__body_information">남성, 177cm, 85kg</p>
        profile_before = soup.find_all('p','review-profile__body_information')
        profile_after = profile_before[i].text.split(',')
        try:
            gender = profile_after[0]
            height = profile_after[1]
            weight = profile_after[2]
        except:
            gender = ''
            height = ''
            weight = ''
#user :<p class="review-profile__name">LV 2 뉴비_95f88e16</p>           
#item: #/<a href="https://www.musinsa.com/app/goods/1231416/0" class="review-goods-information__name">테이퍼드 히든 밴딩 크롭 슬랙스 [더스티 베이지]</a>
#size :<span class="review-goods-information__option">
#content

        try:
            user = soup.find_all('p','review-profile__name')[i].text
            item = soup.find_all('a','review-goods-information__name')[i].text
            # '/n' 없애고 추출하기
            size = soup.find_all('span', 'review-goods-information__option')[i].text.strip().replace('\n','')
            content = soup.find_all('div','review-contents__text')[i].text
        except:
            user = ''
            item = ''
            size = ''
            content = ''
            
        #star
#->별 5개일때:<span class="review-list__rating__active" style="width: 100%"></span>
#->별 4개일때:<span class="review-list__rating__active" style="width: 80%"></span>
        stars = driver.find_elements_by_xpath('//*[@id="reviewListFragment"]/div['+str(i+1)+']/div[3]/span/span/span')
        try:
            for j in stars:
                a =j.get_attribute('style')
                if a[7:9]=='20':
                    star = 1
                elif a[7:9]=='40':
                    star = 2
                elif a[7:9]=='60':
                    star = 3
                elif a[7:9]=='80':
                    star = 4
                else:
                    star = 5
        except:
            star = ''
      

    #evaluation
        evaluation = soup.find_all('div', 'review-evaluation')
        try:
            size_eval = evaluation[i].find_all('span')[0].text
            bright_eval = evaluation[i].find_all('span')[1].text
            color_eval = evaluation[i].find_all('span')[2].text
            thick_eval = evaluation[i].find_all('span')[3].text
        except:
            size_eval = ''
            bright_eval = ''
            color_eval = ''
            thick_eval = ''

        
        user_list.append(user)
        gender_list.append(gender)
        height_list.append(height)
        weight_list.append(weight)
        item_list.append(item)
        size_list.append(size)
        content_list.append(content)
        star_list.append(star)
        size_eval_list.append(size_eval)
        bright_eval_list.append(bright_eval)
        color_eval_list.append(color_eval)
        thick_eval_list.append(thick_eval)
  
   
        
#버튼 누르기 함수정의
def move_next(driver):    
    for p in range(4):
        get_content(driver)
        #페이지 2,3,4,5 넘어가기
        driver.find_element_by_css_selector('#reviewListFragment > div.nslist_bottom > div.pagination.textRight > div > a:nth-child(' + 
                                            str(int(4) + int(p)) + ')').send_keys(Keys.ENTER)
        time.sleep(2)
    get_content(driver)
    
#그다음 화살표'>'버튼누르기: (6,7,8...)있는 페이지로 넘어가기   
def move_arrow(driver):
    driver.find_element_by_css_selector('#reviewListFragment > div.nslist_bottom > div.pagination.textRight > div > a.fa.fa-angle-right.paging-btn.btn.next').send_keys(Keys.ENTER)



In [ ]:
#크롤링 시작
#final_df = pd.DataFrame() 
#8페이지
#options = webdriver.ChromeOptions()
#options.add_experimental_option("excludeSwitches", ["enable-logging"])
#driver = webdriver.Chrome(options=options)

#크롤링 시작[창열기](다정)
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
#가져오고싶은 상품 페이지 링크 입력
url = 'https://www.musinsa.com/categories/item/001004?d_cat_cd=001004&brand=&list_kind=small&sort=emt_high&sub_sort=&page=7&display_cnt=90&group_sale=&exclusive_yn=&sale_goods=&timesale_yn=&ex_soldout=&kids=&color=&price1=&price2=&shoeSizeOption=&tags=&campaign_id=&includeKeywords=&measure='

options = webdriver.ChromeOptions()
options.add_experimental_option('excludeSwitches', ['enable-logging'])
#아래는 알림창 닫는 옵션인데 안먹힘
#options.add_experimental_option('prefs', {'profile.default_content_setting_values.notifications' : 1})

driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
driver.get(url)
time.sleep(2)

In [ ]:
#크롤링 시작[상품정보크롤링](다정)
start = time.time()
math.factorial(100000)

#-----------------item_cnt: 상품개수 + 1로 바꾸기 #90개-> 91------------#
item_cnt = 6
final_df = pd.DataFrame()
#-----------------range(시작아이템개수,,)바꾸기------------------------#
for t in range(1,item_cnt,1):
    #후기순 90개 자동으로 클릭하기
# #searchList > li:nth-child(1) > div.li_inner > div.list_img > a > img
# #searchList > li:nth-child(90) > div.li_inner > div.list_img > a > img
    
    try: 
        driver.find_element_by_css_selector('#searchList > li:nth-child(' +
                                            str(int(t)) +
                                            ') > div.li_inner > div.list_img > a > img').click()
        time.sleep(2)
    except:
        continue
   
    #일단 빈공간 클릭->창이 랜덤으로 뜨기 때문에 빈곳을 클릭하면 팝업이 뜨는 경우가 있음
    driver.find_element_by_xpath('//*[@id="product_order_info"]/div[1]/h4')
    time.sleep(2)
    try:
        #무신사쿠폰 팝업창: 해당 팝업이 뜨면 나머지 선택 안됨
        driver.find_element_by_xpath('/html/body/div/div/div/button').click()
        time.sleep(2)
        #입고지연팝업창: 삭제안해도 나머지 구동 가능
        #driver.find_element_by_xpath('//*[@id="divpop_goods_niceghostclub_8451"]/form/button[2]').click()
        #무신사회원혜택 팝업창 :삭제안해도 나머지 구동가능
        #driver.find_element_by_xpath('//*[@id="page_product_detail"]/div[3]/div[23]/div/a[1]/img').click()
    except:
        pass
        
        #사이즈표
    try:
        html = driver.page_source
        soup = BeautifulSoup(html,'html.parser')
        figure = soup.find('table',{'class':'table_th_grey'})
        p = parser.make2d(figure)
        figure_df = pd.DataFrame(data = p[1:],columns = p[0])
        figure_df.drop([0,1],inplace = True)
    except:
        print("사이즈표 오류발생")
        
        #리뷰개수
    try:
        reviewNum = driver.find_element_by_xpath('//*[@id="estimate_style"]')
        reviewNum = reviewNum.text
        reviewNum = re.sub(r'[^0-9]','',reviewNum)
        reviewNum = int(reviewNum)
        
    except:
        print("리뷰개수 오류발생")
        reviewNum = 0
        

    user_list = []
    gender_list =[]
    height_list = []
    weight_list =[]
    item_list = []
    size_list = []
    star_list= []
    content_list = []
    size_eval_list =[]
    bright_eval_list =[]
    color_eval_list =[]
    thick_eval_list =[]
    
    #여기까진 문제없이 돌아갔음    
    #----------------- b:기준으로 잡은 스타일후기리뷰 개수,우리는 100개------------# 
    b = 100  
    
    if reviewNum >= b:
        #크롤링
        #1:50개, 2:100개, 10:500개, 
        for k in range(2):
            try:
                move_next(driver)
                move_arrow(driver)
        #move_next(driver)
            except:
                time.sleep(2)
    else:
        pass
        

    print(str(t),'번째제품 리뷰',reviewNum,'개')
    
    a = t      
    time.sleep(2)    
    globals()["df" + str(a)] = pd.DataFrame({'user':user_list,
                                           'gender':gender_list,
                                           'height':height_list,
                                           'weight':weight_list,
                                            'item':item_list,
                                            'size':size_list,
                                            'star':star_list,
                                            'content':content_list,
                                            'size_eval':size_eval_list,
                                            'bright_eval':bright_eval_list,
                                            'color_eval':color_eval_list,
                                            'thick_eval':thick_eval_list})          
    #사이즈표와 리뷰 merge
    globals()["merge_df"+str(a)] = pd.merge(globals()["df" + str(a)],figure_df,how = 'left',left_on = 'size',right_on = 'cm')
    print('df',a,'shape',globals()["merge_df"+str(a)].shape)
    
    final_df = pd.concat([final_df, globals()["merge_df"+str(a)]])
    #뒤로가기
    driver.back()
    time.sleep(3)
          


end = time.time()
print(f"{end - start:.5f} sec")

- 5번째 제품의 리뷰가 100개 이상인데, 14개밖에 크롤링 되지 않음

In [ ]:
driver.find_element_by_css_selector('#searchList > li:nth-child(5) > div.li_inner > div.list_img > a > img').click()
time.sleep(2)

In [ ]:
user_list = []
gender_list =[]
height_list = []
weight_list =[]
item_list = []
size_list = []
star_list= []
content_list = []
size_eval_list =[]
bright_eval_list =[]
color_eval_list =[]
thick_eval_list =[]
    
for k in range(2):
    try:
        move_next(driver)
        move_arrow(driver)
    except:
        pass

In [ ]:
user_list

- 따로 창을 열어서 리뷰 크롤링 시도 > 그래도 14개만 크롤링됨
- move_next 함수나 move_arrow 함수 문제?

In [ ]:
#버튼 누르기 함수정의
#그안의 time.sleep을 3초로 변경해줌
def move_next(driver):    
    for p in range(4):
        get_content(driver)
        #페이지 2,3,4,5 넘어가기
        driver.find_element_by_css_selector('#reviewListFragment > div.nslist_bottom > div.pagination.textRight > div > a:nth-child(' + 
                                            str(int(4) + int(p)) + ')').send_keys(Keys.ENTER)
        time.sleep(4)
    get_content(driver)
    
#그다음 화살표'>'버튼누르기: (6,7,8...)있는 페이지로 넘어가기   
def move_arrow(driver):
    driver.find_element_by_css_selector('#reviewListFragment > div.nslist_bottom > div.pagination.textRight > div > a.fa.fa-angle-right.paging-btn.btn.next').send_keys(Keys.ENTER)



In [ ]:
user_list = []
gender_list =[]
height_list = []
weight_list =[]
item_list = []
size_list = []
star_list= []
content_list = []
size_eval_list =[]
bright_eval_list =[]
color_eval_list =[]
thick_eval_list =[]
    
for k in range(2):
    try:
        move_next(driver)
        move_arrow(driver)
    except:
        pass

**창 닫기 시도**
- 창 개수 확인 : 현재 창이 여러개면 제일 처음 창만 두고 닫기 >> 그냥 하나의 창에 뜸 ^.^
- 에러(Alert)창 확인 : 경고창으로 뜨는 경우 닫아주는 함수 존재 >> 이것도 아님 ^.^
- 직접 버튼 지정해서 닫기 : driver.back 함수를 써야하는데, 버튼 닫을때 driver를 건들여서 넘어가지 않음 ^.^